In [3]:
from sympy.parsing.sympy_parser import parse_expr

In [2]:
parse_expr("0.3x + 3y", evaluate=True, local_dict={"x": 10, "y": 3, "z": 5}, transformations="all")

12

In [3]:
# from sympy import Abs
from sympy import *
from sympy.abc import x

parse_expr("max(1, 2, 3)", local_dict={"x": -10}, evaluate=True)

3

In [4]:
from numericals import Numerical

numerical = Numerical()

numerical.summarize_historical_data("AAPL")

[*********************100%***********************]  1 of 1 completed


{'period': '100d',
 'interval': '1d',
 'time_zone': None,
 'average_open': 150.29220054626464,
 'max_open': 173.0500030517578,
 'min_open': 126.01000213623047,
 'stdev_open': 12.932928708865637}

In [7]:
from expression import Expression

expr = Expression(str_expression="3x^2 + average(AAPL_return, 10d, 1m) * average(what, 5d, 1s)", variable_dict={"x": 10})
expr.evaluate("3x^2 + _average(AAPL_adj_close, 10d, 1d) * _average(MSFT_volume, 5d, 1d)")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


3389556823.191317

In [7]:
import re 

str_ = "what(and, now, this, thes(whatever_what, aver_age(now, its, tough), inner(this, is, it) 1d))"

In [8]:
def get_innermost_functions(str_expression, arguments_count):

    # The ",\s*\w+" extracts a single argument. We want to be flexible with the
    # number of arguments we can extract. For instance, if we wnat to extract
    # an inner function that accepts two arguments, the regex for that will
    # be r"\w+\(\w+\s*,\s*\w+\)". That is what we are doing here
    r = "".join([",\s*\w+\s*" for i in range(arguments_count-1)]) + r"\)"
    regex = r"\w+\(\s*\w+\s*" + r

    return re.findall(regex, str_expression)

what = get_innermost_functions(str_, 3)
what

NameError: name 'str_' is not defined

In [2]:
my_arr = ["AAPL", "What"]
what = ["_"+i for i in my_arr]
what

['_AAPL', '_What']

In [7]:
for f in funs:
    print(funs[f][0])

NameError: name 'funs' is not defined

In [ ]:
for f in funs:
    x = expr.evaluate_function(function_=f, expression=funs[f][0])
    # print(funs[f])
    print(x)

# x = funs['average'][0]
# x.replace("average(", "").replace(")", "").replace(" ", "").split(",")


[*********************100%***********************]  1 of 1 completed
Date
2023-04-24     41949600
2023-04-25     48714100
2023-04-26     45498800
2023-04-27     64902300
2023-04-28     55209200
2023-05-01     52472900
2023-05-02     48425700
2023-05-03     65136000
2023-05-04     81235400
2023-05-05    113316400
Name: volume, dtype: int64
61686040.0


In [1]:
from expression import Expression
strr = "_average(AAPL_return, 10d, 1d) + _AAPL(return, 1d)"
expr = Expression(str_expression=strr, portfolio_assets=["AAPL", "MSFT"])

['average(AAPL_return, 10d, 1m)', 'average(what, 5d, 1s)']